In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.animation import PillowWriter
import numba
from numba import jit
from scipy.io import wavfile
from IPython.display import Audio
import os

In [17]:
c = 306
Nx = 101

In [47]:
@numba.jit('f8[:](i8, f8[:,:])')
def integral(n,y):          # is the solution we just get
    sin_arr = np.sin(n*np.linspace(0,np.pi,Nx))
    return np.array([sum(sin_arr*s) for s in y])      #array of relative amplitude for the nth harmonic at all times

C:\Users\张索\AppData\Local\Temp\ipykernel_28024\1941127412.py:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit('f8[:](i8, f8[:,:])')


In [6]:
@numba.jit("f8[:,:](f8[:,:], f8, f8, f8, f8)", nopython=True, nogil=True)
def compute(y, dt, dx, l, gamma):
    outer_factor = 1/(1/(c * dt)**2+gamma/2/dt)
    times = len(y)
    length = len(y[0])
    for m in range(1,times-1):
        for j in range(2,length-2):
            f = (y[m][j+1]-2*y[m][j]+y[m][j-1])/dx**2
            g = -(y[m-1][j]-2*y[m][j])/(c**2 * dt**2)
            h = gamma * y[m-1][j] / 2/dt
            i = - (l/dx**2)**2 * (y[m][j-2]-4*y[m][j-1]+6*y[m][j]-4*y[m][j+1]+y[m][j+2])
            y[m+1][j] = outer_factor * (f+g+h+i)
    return y

In [53]:
def produce_note(strLength, path, name):
    L = strLength
    dx = L/(Nx-1)

    duration = 3
    dt = 2e-06
    Nt = int(duration / dt)

    l=2e-6
    gamma=2.6e-5

    ya = np.linspace(0, 0.01, 80)
    yb = np.linspace(0.01, 0, Nx-80)
    y0 = np.concatenate([ya,yb])
    sol = np.zeros((Nt,Nx))
    sol[0] = y0
    sol[1] = y0
    sol = compute(sol, dt, dx, l, gamma)
    numberoHarmonics = 10
    hms = np.array([integral(n,sol) for n in range(1,numberoHarmonics+1)])
    t = np.linspace(0,duration,Nt)
    tot = np.array(sum(hms[i]*np.sin((i+1)*np.pi*c*t/L) for i in range(numberoHarmonics)))
    tot = tot[::10].astype(np.float32)     # 50k, usual mp4 needs 48 kHz
    loc = os.path.join(path,f'{name}.wav')
    wavfile.write(loc,50000,tot)
    